In [1]:
%display latex

import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [2]:
eps = var('eps', latex_name = '\\varepsilon')
t = var('t')
r = var('r')
th = var('th', latex_name = '\\theta')
ph = var('ph', latex_name = '\\varphi')

coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega', latex_name='\\omega')(r,th)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Omegae = function('Omegaevalue')(eps)
Ne = function('Ne', latex_name='N_e')(r,eps)

In [3]:
def cosmotensors(gedd):
    
    geuu = 1/gedd
    
    chris = [[[0 for k in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                chris[i][j][k] = 0.5*sum(geuu[i,dd]*(diff(gedd[dd,j],coords[k])
                                                   + diff(gedd[dd,k],coords[j])
                                                   - diff(gedd[j,k],coords[dd])) for dd in range(0,4))
                
    riem = [[[[0 for l in range(0,4)] for k in range(0,4)] for j in range(0,4)] for i in range(0,4)]
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                for l in range(0,4):
                    riem[i][j][k][l] = (diff(chris[i][j][l],coords[k])
                                      - diff(chris[i][j][k],coords[l])
                                      + sum(chris[i][k][dd]*chris[dd][l][j]
                                          - chris[i][l][dd]*chris[dd][k][j] for dd in range(0,4)))
                    
    ricci = matrix(SR, 4, 4)
    for i in range(0,4):
        for k in range(0,4):
            ricci[i,k] = sum(riem[dd][i][dd][k] for dd in range(0,4))

    s_curv = sum(sum(ricci[i,j]*geuu[i,j] for i in range(0,4)) for j in range(0,4))
    
    Ge = matrix(SR, 4, 4)
    Ge = ricci - 0.5*gedd*s_curv
    
    dGe = matrix(SR, 4, 4)
    ddGe = matrix(SR, 4, 4)
    for i in range(0,4):
        for j in range(0,4):
            dGe[i,j] = diff(Ge[i,j],eps)
            ddGe[i,j] = diff(dGe[i,j],eps)
    
    return {'inverse_metric': geuu,
            'einstein': Ge,
            'd_einstein': dGe,
            'dd_einstein': ddGe}

In [ ]:
gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(i,4):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]
        gedd[j,i] = gedd[i,j]

tensors = cosmotensors(gedd)

geuu = tensors['inverse_metric']
Ge = tensors['einstein']
dGe = tensors['d_einstein']
ddGe = tensors['dd_einstein']

In [ ]:
show("ge0 = ", gdd[:])
show("K1 = ", K1[:])
show("K2 = ", K2[:])
show("ge = ", gedd[:])

In [ ]:
#4-velocities of the fluid for any epsilon

ue = [0,0,0,0]
u = [0,0,0,0]

ue[0] = Ne;         ue[1] = 0;   ue[2] = 0;   ue[3] = Ne*Omegae
u[0] = exp(-nu/2);  u[1] =  0;   u[2] = 0;    u[3] =  0

Nevalue = solve(u[0]==ue[0], Ne)[0].rhs()                                                             #Eq 4.20
Omegaevalue = solve(u[3]==ue[3], Omegae)[0].rhs()                                                     #Eq 4.20

In [ ]:
show(LatexExpr("N_{\\varepsilon}\lvert_0 = "), Nevalue)
show(LatexExpr("\\Omega_{\\varepsilon}\lvert_0 = "), Omegaevalue)

In [ ]:
u1 = [0,0,0,0]
u2 = [0,0,0,0]

u10 = function('u10',latex_name="u^{(1)0}")(r)
u20 = function('u20',latex_name="u^{(2)0}")(r)

Omega1 = var('Omega1', latex_name="\\Omega^{(1)}")
Omega2 = var('Omega2', latex_name="\\Omega^{(2)}")

s0 = eps == 0

s1 = Ne.subs(eps==0) == Nevalue
s2 = Omegae.subs(eps==0) == Omegaevalue

s3 = diff(Ne,eps,eps).subs(eps==0) == u20
s4 = diff(Omegae,eps,eps).subs(eps==0) == Omega2

s5 = diff(Ne,eps).subs(eps==0) == u10
s6 = diff(Omegae,eps).subs(eps==0) == Omega1

sustituciones = [s0,s1,s2,s3,s4,s5,s6]

for i in range(0,4):
    u1[i] = diff(ue[i],eps)
    u2[i] = diff(ue[i],eps,eps)

Ue = [0,0,0,0]
for i in range(0,4):
    for j in range(0,4):
        Ue[i] += ue[j]*gedd[j,i]

geUeue = 0
for i in range(0,4):
    geUeue += Ue[i]*ue[i]

ddgeUeue = diff(geUeue, eps, eps).subs(sustituciones) == 0
u20value = solve(ddgeUeue, u20)[0].rhs()
s7 = u20 == u20value

dgeUeue = diff(geUeue, eps).subs(sustituciones) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s8 = u10 == u10value

sustituciones.extend([s7,s8])
for i in range(0,4):
    for k in range(0,len(sustituciones)):
        u1[i] = u1[i].subs(sustituciones[k])
        u2[i] = u2[i].subs(sustituciones[k])

In [ ]:
show(LatexExpr("u^{(1)} = "), u1)                                                                         #Eq 4.21
show(LatexExpr("u^{(2)} = "), u2)                                                                         #Eq 4.22

In [ ]:
# CHECK
u1_paper = [0,0,0,0]
u1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
u1_paper[3] = e^(-nu/2)*Omega1

u2_paper = [0,0,0,0]
u2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omega1*K1[0,3]+Omega1^2*r^2*sin(th)^2)
u2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omega1+Omega2)  

u1_sum = 0
u2_sum = 0
for i in range(0,4):
    u1_sum += u1[0] - u1_paper[0]
    u2_sum += u2[0] - u2_paper[0]

u1_sum = u1_sum.simplify_full()
u2_sum = u2_sum.simplify_full()

show(u1_sum)
show(u2_sum)

In [ ]:
G = copy(Ge)
for k in range(0,len(sustituciones)):
    G = G.subs(sustituciones[k])

In [ ]:
Ee = function('Ee', latex_name='E_{\\varepsilon}')(r,th,eps)
Pe = function('Pe', latex_name='P_{\\varepsilon}')(r,th,eps)

Te = matrix(SR, 4, 4)
dTe = matrix(SR, 4, 4)
ddTe = matrix(SR, 4, 4)
for i in range(0,4):
    for j in range(0,4):
        Te[i,j] = (Ee+Pe)*Ue[i]*Ue[j]+Pe*gedd[i,j]
        dTe[i,j] = diff(Te[i,j],eps)
        ddTe[i,j] = diff(Te[i,j],eps,eps)

T = copy(Te)
for k in range(0,len(sustituciones)):
    T = T.subs(sustituciones[k])

In [ ]:
show(LatexExpr("T_{\\varepsilon}\lvert_0 ="), T)

In [ ]:
kappa = 8*pi
E = function('E')(r,th)
P = function('P')(r,th)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sustituciones.extend([s9,s10])

T = T.subs(s9).subs(s10)
        
Evalue = solve(G[0,0]==kappa*T[0,0],E)[0].rhs()                                                          #Eq 3.7
Pvalue = solve(G[1,1]==kappa*T[1,1],P)[0].rhs()                                                          #Eq 3.8
Pvalue2 = solve(G[2,2]==kappa*T[2,2],P)[0].rhs()
Pvalue3 = solve(G[3,3]==kappa*T[3,3],P)[0].rhs()
ddnu = solve(Pvalue == Pvalue2, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [ ]:
show(LatexExpr("E = "), Evalue)
show(LatexExpr("P_{rr} = "), Pvalue)
show(LatexExpr("\\nu'' ="), ddnu)

s11 = E == Evalue
s12 = P == Pvalue
s13 = diff(nu,r,r) == ddnu
sustituciones.extend([s11,s12,s13])

In [ ]:
dlamb = solve(E-Evalue==0,diff(lamb,r))[0].rhs()                                                              #Eq 3.10
dnu = solve(P-Pvalue==0,diff(nu,r))[0].rhs()                                                                  #Eq 3.11

In [ ]:
show(LatexExpr("\\lambda' = "), dlamb)
show(LatexExpr("\\nu' = "), dnu)

In [ ]:
h0 = function('h0', latex_name='h_0')(r,th)
k0 = function('k0', latex_name='k_0')(r,th)
m0 = function('m0', latex_name='m_0')(r,th)

h1 = function('h1', latex_name='h_1')(r,th)
k1 = function('k1', latex_name='k_1')(r,th)
m1 = function('m1', latex_name='m_1')(r,th)

h2 = function('h2', latex_name='h_2')(r,th)
k2 = function('k2', latex_name='k_2')(r,th)
m2 = function('m2', latex_name='m_2')(r,th)

LP1 = function('LP1', latex_name='LP_1')(th)
LP2 = function('LP2', latex_name='LP_2')(th)

E1 = function('E1', latex_name='E^{(1)}')(r,th)
E2 = function('E2', latex_name='E^{(2)}')(r,th)
P1 = function('P1', latex_name='P^{(1)}')(r,th)
P2 = function('P2', latex_name='P^{(2)}')(r,th)

E02 = function('E02', latex_name='E_{0}^{(2)}')(r,th)
E12 = function('E12', latex_name='E_{1}^{(2)}')(r,th)
E22 = function('E22', latex_name='E_{2}^{(2)}')(r,th)

P02 = function('P02', latex_name='P_{0}^{(2)}')(r,th)
P12 = function('P12', latex_name='P_{1}^{(2)}')(r,th)
P22 = function('P22', latex_name='P_{2}^{(2)}')(r,th)

## Primera derivada

In [ ]:
s14 = diff(Ee,eps).subs(eps==0) == E1
s15 = diff(Ee,eps,eps).subs(eps==0) == E2
s16 = diff(Pe,eps).subs(eps==0) == P1
s17 = diff(Pe,eps,eps).subs(eps==0) == P2

sustituciones.extend([s14,s15,s16,s17])

In [ ]:
dG = copy(dGe)
dT = copy(dTe)
for k in range(0,len(sustituciones)):
    dG = dG.subs(sustituciones[k])
    dT = dT.subs(sustituciones[k])

In [ ]:
dEq11 = dG[1,1] - kappa*dT[1,1] == 0
dEq11 = dEq11.simplify_full()

dEq22 = dG[2,2] - kappa*dT[2,2] == 0
dEq22 = dEq22.simplify_full()

dEq33 = dG[3,3] - kappa*dT[3,3] == 0
dEq33 = dEq33.simplify_full()

show(dEq11)
show(dEq22)
show(dEq33)

P1value = solve(dEq11,P1)[0].rhs()
sustituciones.extend([P1==P1value])

show(LatexExpr("\\text{De aquí sacamos }P_1(r) \\rightarrow P_1(r) = "), P1value)

In [ ]:
dEq00 = dG[0,0] - kappa*dT[0,0] == 0
dEq00 = dEq00.simplify_full()
show(dEq00)

for k in range(0,len(sustituciones)):
    dEq00 = dEq00.subs(sustituciones[k])

E1value = solve(dEq00,E1)[0].rhs()
sustituciones.extend([E1==E1value])

show(LatexExpr("\\text{De aquí sacamos }E_1(r,\\theta) \\rightarrow E_1(r,\\theta) = "), E1value)

In [ ]:
#DEMOSTRAR QUE ESTO ES LA ECUACIÓN 48
dEq03 = dG[0,3] - kappa*dT[0,3]

for k in range(0,len(sustituciones)):
    dEq03 = dEq03.subs(sustituciones[k])
    
dEq03 = dEq03.simplify_full()
dEq03                                                                                                       #Eq 48

In [ ]:
# CHECK
jj = e^(-(lamb+nu)/2)
dEq03_paper = diff(r^4*jj*diff(omega,r),r)+\
              (r^2*jj*e^(lamb)/sin(th)^3)*diff(sin(th)^3*diff(omega,th),th)+\
              4*r^3*diff(jj,r)*(omega-Omega1)

dEq03_paper = dEq03_paper*sin(th)^2/(2*r^2*e^(lamb/2-nu/2))

dEq03_check = dEq03_paper - dEq03

dEq03_check.simplify_full()

## Segunda derivada

In [ ]:
ddG = copy(ddGe)
ddT = copy(ddTe)
for k in range(0,len(sustituciones)):
    ddG = ddG.subs(sustituciones[k])
    ddT = ddT.subs(sustituciones[k])

In [ ]:
# Aplicamos barotropic EoS
# Utilizando (6.11) de Reina con P1=0 y E1=0
#CAMBIAR E2 POR E^(2) Y APLICARLO EN ALGÚN MOMENTO
E2*diff(P,r) - P2*diff(E,r) == 0

### Descomposición en polinomios de Legendre

In [ ]:
sustituciones = [m==m0+m1*LP1+m2*LP2,
                 h==h0+h1*LP1+h2*LP2,
                 k==k0+k1*LP1+k2*LP2,
                 E2==E02+E12*LP1+E22*LP2,
                 P2==P02+P12*LP1+P22*LP2]

for k in range(0,len(sustituciones)):
    ddG = ddG.subs(sustituciones[k])
    ddT = ddT.subs(sustituciones[k])

show(ddT[0,0].simplify_full())

In [ ]:
s1 = sin(th)^2 == (2-2*LP2)/3
s2 = 1/sin(th)^2 == 3/(2-2*LP2)
s3 = cos(th) == LP1
s4 = cos(th)^2 == (2*LP2+1)/3
s5 = LP1^2 == (2*LP2+1)/3
s6 = diff(LP1, th, th) == -LP1
s7 = diff(LP2, th) == -3*LP1*sin(th)
s8 = diff(LP2, th, th) == 1-4*LP2

sustituciones = [s1,s2,s3,s4,s5,s6,s7,s8]

In [ ]:
for k in range(0,len(sustituciones)):
    ddG = ddG.subs(sustituciones[k])
    ddT = ddT.subs(sustituciones[k])

In [ ]:
ddG = ddG.subs(LP1==0).subs(LP2==0)
ddT = ddT.subs(LP1==0).subs(LP2==0)

In [ ]:
ddEq00 = ddG[0,0] - kappa*ddT[0,0] == 0
E02value = solve(ddEq00, E02)[0].rhs()

E02value_paper = ((4/r^2)*diff(r*e^(-lamb)*m0,r)\
                  +8/3*r*jj*diff(jj,r)*(omega-Omega1)^2\
                  -1/3*jj^2*r^2*diff(omega,r)^2)/(8*pi)

for k in range(0,len(sustituciones)):
    E02value_paper = E02value_paper.subs(sustituciones[k])

E02value_check = (E02value - E02value_paper)
E02value_check = E02value_check.simplify_full()
E02value_check

In [ ]:
ddEq11 = ddG[1,1] - kappa*ddT[1,1] == 0
P02value = solve(ddEq11, P02)[0].rhs()

P02value_paper = ((4/r^2)*(e^(-lamb)*r*diff(h0,r)\
                  -m0*(8*pi*r^2*P+1))\
                  +1/3*r^2*jj^2*diff(omega,r)^2)/(8*pi)

P02value_check = (P02value - P02value_paper)
P02value_check = P02value_check.simplify_full()
P02value_check

In [ ]:
ddEq03 = ddG[0,3] - kappa*ddT[0,3] == 0
ddEq03 = ddEq03*(3/(16*pi*r^2*Omega2))
ddEq03 = ddEq03.simplify_full()

ddEq12 = ddG[1,2] - kappa*ddT[1,2] == 0
ddEq12 = ddEq12*(r*e^(nu))
ddEq12 = ddEq12.simplify_full()